In [1]:
# DON'T CHANGE this part: import libraries
import numpy as np
import scipy
import json
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import itertools

In [2]:
# DON'T CHANGE this part: read data path
train_set_path, valid_set_path, random_number = input().split()

train.json valid.json 10


In [3]:
# TODO:
# 1. preprocess: converting text to lowercase, coverting number, tokenization, removing stopword, stemming
# 2. embedding: hitogram matrix
# 3. classifier using linear regression
# 4. accuracy (for metric)
# Mình làm theo 4 bước ở đây

In [4]:
print(train_set_path)
print(valid_set_path)
print(random_number)


train.json
valid.json
10


In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
# đọc file json
def read_json(data_path):
    overall_list = []
    review_list = []
    f = open(data_path)
    data = json.load(f) 
    for item in data: 
        overall_list.append(item['overall'])
        review_list.append(item['reviewText'])
    return review_list,overall_list


X_train,y_train = read_json(train_set_path)
print(len(X_train),len(y_train))

print(X_train[0])
print(y_train[0])

#Dữ liệu sau khi chạy hai hàm print ở trên cho tập train X và tập train Y
# 5000 5000
# I needed SHORT cables to connect adjacent units in a rack.  This was a perfect fit.  The colors make signal tracing a matter of sight rather than  scribble strips.
# 5.0

5000 5000
I needed SHORT cables to connect adjacent units in a rack.  This was a perfect fit.  The colors make signal tracing a matter of sight rather than  scribble strips.
5.0


In [21]:
X_valid,y_valid = read_json(valid_set_path)

#print(len(X_valid),len(y_valid))
#print(X_valid[0])
#print(y_valid[0])

#--lấy ra giá trị X và giá trị Y thông qua hàm đọc file json
# print(len(X_valid),len(y_valid)) -- 500 500
# print(X_valid[0]) -- These pens never get old. I prefer medium point. LOVE the mulitple colors.
# print(y_valid[0]) -- 5.0

500 500
These pens never get old. I prefer medium point. LOVE the mulitple colors.
5.0


#### Ví dụ cho phần báo cáo, nên báo cáo cho từng phần code để rõ ràng

Báo cáo phần tiền xử lý: dùng xyz để tách từ, ...

... Đối với những từ out-of-vocab (xuất hiện trong tập train nhưng không có ở tập valid), xử lý bằng cách ... 

In [23]:
# Ví dụ cho phần code hàm tiền xử lý tập train
# 1. preprocess: converting text to lowercase, coverting number, tokenization, removing stopword, stemming
# Đầu vào là 1 văn bản, đầu ra văn bản đã được tiền xử lý sử dụng các bước ở trên.
stemmer= PorterStemmer()
stop_words = set(stopwords.words("english"))
def preprocess_train(text): #code này được dùng để làm j, tham số truyền vào là gì?
    # converting text to lowercase
    text = text.lower()
    # Chuyển giá trị số về num
    text = re.sub(r"[0-9]+", " num ", text)
    # Tokenizer
    tokens = word_tokenize(text)
    # xoá stopword
    text = [i for i in tokens if not i in stop_words]
    # Stemming
    text = [stemmer.stem(word) for word in text]
    return text

def preprocess(X_data):
    list_new = []
    for x in X_data:
        x_processed = preprocess_train(x)
        list_new.append(x_processed)
    return list_new

pre_text = preprocess(X_train)
#print(pre_text[int(random_number)])


In [9]:
# Lấy toàn bộ vocabulary trong tập train để kiếm tra trong tập test và thay bằng giá trị unk
vocab = ["unk"]
for item in pre_text:
    for token in item:
        if token not in vocab:
            vocab.append(token)
#print("Số lượng từ vựng trong tập train: ", len(vocab))
#print(vocab): các từ trong tập train

Số lượng từ vựng trong tập train:  6523
['unk', 'need', 'short', 'cabl', 'connect', 'adjac', 'unit', 'rack', '.', 'perfect', 'fit', 'color', 'make', 'signal', 'trace', 'matter', 'sight', 'rather', 'scribbl', 'strip', 'first', 'set', 'copic', 'got', ',', "'s", 'great', 'pack', 'skinton', ':', ')', 'love', 'stencil', 'beauti', 'result', '...', 'good', 'gelli', 'plate', 'print', 'price', 'product', 'custom', 'servic', 'energel', 'pen', 'perfectli', '!', 'save', 'money', 'get', 'refil', 'instead', 'whole', 'gener', "'re", 'resistor', 'chang', 'resist', 'light', 'read', 'review', '?', 'blade', 'develop', 'nick', 'almost', 'immedi', 'take', 'care', 'might', 'defect', 'get-go', 'rate', 'buy', 'anoth', 'nice', 'select', 'famili', 'use', 'top', 'four', 'togeth', 'bottom', 'packag', 'one', 'bright', 'sparkli', 'size', 'realli', 'lot', 'want', 'brush', 'holder', 'angl', 'design', 'terrif', 'work', 'ok', 'keep', 'fall', 'mous', 'go', 'wire', 'wireless', 'eas', 'lag', 'highli', 'recommend', 'thank'

In [10]:
#print(X_valid) --['These pens never get old. I prefer medium point. LOVE the mulitple colors.', "Not as strong as I would have liked as is my Peavey mike but they'll do for now for the price.", 'Great calendar but the bottom edge of the sheets will wrinkle and fray with use. If there was a protective bottom edge it would make this a great product.',...]

['These pens never get old. I prefer medium point. LOVE the mulitple colors.', "Not as strong as I would have liked as is my Peavey mike but they'll do for now for the price.", 'Great calendar but the bottom edge of the sheets will wrinkle and fray with use. If there was a protective bottom edge it would make this a great product.', "One of the best RPG's I've played since Final Fantasy 6. It's clever, challenging, well written, and captivating. You will not regret this buy.", 'Still a great gaming system. For those of you who just started playing games on the 3DS, it takes getting used to the the 3D effects for some of the games but the games can still be enjoyed without the 3D effects.', "This was a great birthday present my son won't stop playing it for two weeks.  I had to take the game from him.", "These canvases are fantastic!  A friend of mine loves to paint so I got her some paints and these canvases.  She loves giving away paintings as gifts and these are the perfect size.  Pl

In [11]:
# 1. preprocess: converting text to lowercase, coverting number, tokenization, removing stopword, stemming
# Cái này xử lý trên tập test. Bởi vì trong nội dung video cô kêu là các từ không xuất hiện trong tập train thì bằng giá trị unk.
# Do đó, chúng ta cần xây dựng 1 hàm riêng với đầu vào là 1 văn bản và tập từ điển trong tập train để kiểm tra xem cái từ trong tập test có tồn tại trong 
# trong từ điển tập train không. Nếu tồn tại thì giữ nguyên. Không tồn tại thì thay bằng giấ trị "unk"
def preprocess_testData(text,vocab):
    # converting text to lowercase
    text = text.lower()
    # Chuyển giá trị số về num
    text = re.sub(r"[0-9]+", " num ", text)
    # Tokenizer
    tokens = word_tokenize(text)
    # Remove stopword
    text = [i for i in tokens if not i in stop_words]
    # Stemming
    text = [stemmer.stem(word) for word in text]
    # chuyển chữ trong tập test thành giá trị unk
    list_new = []
    for token in text:
        if token in vocab:
            list_new.append(token)
        else:
            list_new.append("unk")
    text = list_new
    return text

# Hàm trên là xử lý ttừng văn bản. Còn hàm này là duyệt toàn bộ dữ liệu trong tập valid hoặc tập test.
# Danh sách là trả về một list.
def preprocess_test(X_valid,vocab): 
    list_process = []
    for item in X_valid:
        x_processed = preprocess_testData(item,vocab)
        list_process.append(x_processed)
    return list_process

pre_text_valid = preprocess_test(X_valid,vocab)
print(pre_text_valid[int(random_number)]) #in câu thứ random_number của tập valid (từ valid_set_path)

['brought', 'back', 'num', 'player', 'mode', 'like', 'mario', 'parti', 'num', '&', 'num']


In [12]:
# câu 2 . Histogram. Đầu vào là 1 văn bản, đầu ra là vector histogram. Document embedding đó.
# Ta xem lại cái công thức histogram vector trong slide
def histogram_vector(document,vocab): #hàm này dc dùng để làm j á, tham số truyền vào
    vector = np.zeros(len(vocab))
    vector_one = np.ones(len(vocab))
    for index,word in enumerate(vocab):
        if word in document:
            vector[index] = document.count(word)
    histogram = vector/(vector_one.T @ vector)
    return histogram
# Ví dụ
s = "i love you baby baby".split(' ')
histogram_vector(s,["i","love","baby"])

array([0.25, 0.25, 0.5 ])

In [13]:
#print(pre_text[0])

['need', 'short', 'cabl', 'connect', 'adjac', 'unit', 'rack', '.', 'perfect', 'fit', '.', 'color', 'make', 'signal', 'trace', 'matter', 'sight', 'rather', 'scribbl', 'strip', '.']


In [14]:
# Tính giá trị histogram cho toàn bộ tập dữ liệu. Đầu vào là danh sách dữ liệu từng tập. Đầu ra là matrận histogram document cho toàn bộ dữ liệu
def matrix_histogram(X_data,vocab):
    matrix = []
    for item in X_data:
        matrix.append(histogram_vector(item,vocab))
    return matrix

histogram_X_train = matrix_histogram(pre_text,vocab)
histogram_X_valid = matrix_histogram(pre_text_valid,vocab)

In [15]:
# Hiện thị ví dụ document embedding (histogram vector)
# print(histogram_X_train[0]) --[0.         0.04761905 0.04761905 ... 0.         0.         0.        ]
# Hiện thị độ dài của 1 vector hisgram. Nó là chiều của tập từ điển vocab. Xem video trình bày Cô sẽ hiểu cái.
# print(len(histogram_X_train[0])) --6523

[0.         0.04761905 0.04761905 ... 0.         0.         0.        ]
6523


In [16]:
# chuyển nhãn thành vector
def category_vector(y):
    vector = np.zeros(5)
    vector[int(y)-1] = 1.0
    return vector

# Hàm chuyển toàn bộ nhãn y trong tập train và tập valid thành vector 5 chiều.
# Ví dụ nhãn 1.0 sẽ thành [1,0,0,0,0], nhãn 5.0 thành [0,0,0,0,1]
def convert_y(y_data):
    matrix = []
    for item in y_data:
        matrix.append(category_vector(item))
    return matrix
y_train_encode = convert_y(y_train)
y_valid_encode = convert_y(y_valid)
# Hiện thị ví dụ ở đây.
# print(y_train[0]) --5.0
# print(y_train_encode[0]) --[0. 0. 0. 0. 1.]

5.0
[0. 0. 0. 0. 1.]


In [17]:
# Hàm linear regression như ở lab 4. Đầu vào là x_train, y_train. đầu ra là giá trị theta
def linear_regression(X_train,y_train):
    theta = np.linalg.pinv(X_train) @ y_train
    return theta
# Huấn luyện trên dữ liệu X_train
theta = linear_regression(histogram_X_train,y_train_encode)
#print("theta : ", theta)
#print(len(theta)) --6523

# theta :  [[ 9.86483024e-12  5.51049173e-11 -2.29972476e-11 -3.01103150e-11
#   -1.15498186e-11]
#  [ 4.17777377e-02 -6.15464223e-02 -1.64461538e-01  2.43738281e-02
#    1.15985639e+00]
#  [ 1.68808765e+00  1.43154843e-01 -1.00180007e-01  3.50076476e+00
#   -4.23182724e+00]
#  ...
#  [ 7.52603474e+00  9.98995941e+00 -9.06672293e+00  3.36612294e+01
#   -4.08743587e+01]
#  [-6.74111785e+00  2.03874042e+01  2.57644048e+01 -3.58669515e+01
#   -2.94373973e+00]
#  [-1.65997390e+00  3.36241697e-01  2.66890431e+00  1.05087079e+01
#   -1.08538800e+01]]

theta :  [[ 9.86483024e-12  5.51049173e-11 -2.29972476e-11 -3.01103150e-11
  -1.15498186e-11]
 [ 4.17777377e-02 -6.15464223e-02 -1.64461538e-01  2.43738281e-02
   1.15985639e+00]
 [ 1.68808765e+00  1.43154843e-01 -1.00180007e-01  3.50076476e+00
  -4.23182724e+00]
 ...
 [ 7.52603474e+00  9.98995941e+00 -9.06672293e+00  3.36612294e+01
  -4.08743587e+01]
 [-6.74111785e+00  2.03874042e+01  2.57644048e+01 -3.58669515e+01
  -2.94373973e+00]
 [-1.65997390e+00  3.36241697e-01  2.66890431e+00  1.05087079e+01
  -1.08538800e+01]]
6523


In [18]:
# Hàm softmax vì ở đây mình làm câu 5 nhãn (Mô hình M2. 10đ)
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)


In [19]:
# Sử dụng hàm softmax trước khi xác định nhãn.
def predict(theta,input):
    predict_value = np.dot(input, theta)
    output = softmax(predict_value)
    label = np.where(output == np.amax(output))[0][0] + 1
    return label

# Chạy dự đoán trên tập test và lưu vào danh sách.
y_valid = [int(x) for x in y_valid]
y_pred = []
for index in range(len(histogram_X_valid)):
    y_pred.append(predict(theta,histogram_X_valid[index]))
# hàng ở trên là nhãn dự đoán của mô hình linear
# Hàng ở dưới là nhãn đúng của nó.

#print(y_pred) --[4, 5, 4, 4, 5, 5, 5, 1, 5, 4, 5, 4, 5, 5, 4, 5, 5, 4, 3, 5, 5, 5, 2, 3, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 3, 5, 2, 5, 5, 3, 5, 5, 4, 5, 4, 4, 4, 5, 5, 5, 5, 4, 5, 5, 4, 5, 3, 5, 5, 5, 5, 3, 3, 5, 4, 5, 3, 5, 5, 4, 4, 5, 5, 3, 3, 1, 5, 1, 3, 5, 4, 5, 3, 4, 4, 5, 4, 5, 4, 5, 4, 5, 3, 4, 4, 5, 5, 4, 4, 5, 2, 4, 5, 3, 5, 5, 5, 1, 5, 5, 2, 5, 4, 5, 5, 5, 4, 2, 5, 5, 4, 5, 5, 5, 5, 4, 4, 5, 2, 5, 5, 5, 4, 4, 5, 4, 5, 5, 1, 1, 1, 4, 5, 5, 5, 3, 1, 5, 3, 2, 3, 5, 5, 4, 5, 5, 4, 4, 5, 5, 5, 4, 5, 2, 5, 2, 5, 5, 5, 5, 5, 5, 4, 5, 4, 2, 5, 3, 1, 3, 5, 3, 3, 5, 4, 4, 5, 5, 1, 5, 5, 3, 5, 5, 5, 3, 3, 4, 4, 5, 5, 5, 3, 4, 2, 5, 4, 4, 3, 5, 1, 5, 4, 5, 4, 5, 2, 5, 4, 5, 5, 5, 5, 5, 4, 5, 5, 4, 4, 5, 5, 4, 5, 5, 4, 5, 5, 5, 4, 5, 1, 4, 5, 5, 3, 5, 5, 4, 5, 4, 5, 5, 5, 4, 5, 4, 3, 5, 5, 5, 4, 5, 5, 5, 4, 5, 4, 4, 5, 5, 3, 4, 5, 5, 3, 5, 4, 5, 5, 2, 5, 5, 5, 4, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4, 5, 3, 5, 1, 3, 5, 5, 3, 4, 2, 3, 5, 5, 5, 4, 5, 3, 5, 4, 5, 3, 5, 5, 2, 5, 4, 5, 5, 5, 5, 2, 5, 2, 4, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 2, 5, 5, 1, 5, 5, 5, 5, 5, 4, 4, 3, 2, 5, 5, 5, 4, 4, 5, 5, 5, 3, 3, 3, 4, 2, 4, 5, 2, 2, 5, 5, 5, 4, 5, 5, 5, 5, 3, 4, 2, 1, 3, 5, 5, 5, 3, 5, 4, 5, 5, 5, 5, 5, 5, 4, 5, 4, 3, 4, 4, 5, 5, 4, 5, 5, 5, 5, 3, 3, 4, 5, 5, 3, 5, 5, 5, 5, 3, 4, 5, 5, 5, 4, 4, 5, 1, 5, 5, 5, 5, 5, 5, 3, 5, 2, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, 4, 5, 4, 5, 5, 4, 1, 1, 5, 5, 5, 5, 5, 3, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 3, 4, 5, 1, 3, 3, 3, 5, 1, 5, 3, 5, 5, 5, 5, 4, 4, 5, 5]
#print(y_valid) -- [5, 3, 3, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 1, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 4, 4, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 1, 5, 4, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 3, 5, 3, 5, 5, 4, 5, 1, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 4, 5, 5, 3, 5, 5, 5, 5, 4, 5, 5, 5, 5, 2, 4, 5, 1, 1, 5, 5, 5, 5, 2, 5, 4, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 3, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 2, 5, 2, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 5, 5, 5, 4, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 4, 5, 4, 5, 5, 1, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 3, 2, 5, 5, 1, 1, 5, 5, 5, 5, 5, 4, 2, 5, 4, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 1, 4, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 5, 5, 4, 4, 3, 3, 3, 5, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1, 2, 5, 5, 3, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 4, 5, 5, 5, 5, 5, 1, 4, 3, 5, 5, 5, 5, 5, 5, 3, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 4, 5, 4, 1, 4, 4, 5, 5, 5, 5, 3, 1, 5, 5, 1, 5, 4, 4, 3, 4, 3, 5, 5, 5, 3, 5, 3, 5, 5, 5, 5, 5, 5]

[4, 5, 4, 4, 5, 5, 5, 1, 5, 4, 5, 4, 5, 5, 4, 5, 5, 4, 3, 5, 5, 5, 2, 3, 5, 5, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 3, 5, 2, 5, 5, 3, 5, 5, 4, 5, 4, 4, 4, 5, 5, 5, 5, 4, 5, 5, 4, 5, 3, 5, 5, 5, 5, 3, 3, 5, 4, 5, 3, 5, 5, 4, 4, 5, 5, 3, 3, 1, 5, 1, 3, 5, 4, 5, 3, 4, 4, 5, 4, 5, 4, 5, 4, 5, 3, 4, 4, 5, 5, 4, 4, 5, 2, 4, 5, 3, 5, 5, 5, 1, 5, 5, 2, 5, 4, 5, 5, 5, 4, 2, 5, 5, 4, 5, 5, 5, 5, 4, 4, 5, 2, 5, 5, 5, 4, 4, 5, 4, 5, 5, 1, 1, 1, 4, 5, 5, 5, 3, 1, 5, 3, 2, 3, 5, 5, 4, 5, 5, 4, 4, 5, 5, 5, 4, 5, 2, 5, 2, 5, 5, 5, 5, 5, 5, 4, 5, 4, 2, 5, 3, 1, 3, 5, 3, 3, 5, 4, 4, 5, 5, 1, 5, 5, 3, 5, 5, 5, 3, 3, 4, 4, 5, 5, 5, 3, 4, 2, 5, 4, 4, 3, 5, 1, 5, 4, 5, 4, 5, 2, 5, 4, 5, 5, 5, 5, 5, 4, 5, 5, 4, 4, 5, 5, 4, 5, 5, 4, 5, 5, 5, 4, 5, 1, 4, 5, 5, 3, 5, 5, 4, 5, 4, 5, 5, 5, 4, 5, 4, 3, 5, 5, 5, 4, 5, 5, 5, 4, 5, 4, 4, 5, 5, 3, 4, 5, 5, 3, 5, 4, 5, 5, 2, 5, 5, 5, 4, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 4, 5, 3, 5, 1, 3, 5, 5, 3, 4, 2, 3, 5, 5, 5, 4, 5, 3, 5, 4, 5, 3, 5, 5, 2, 5, 4, 5, 5, 5, 5, 2, 

In [20]:
# Tính độ chính xác * 100%.
def accuracy_score(y_pred, y_test):
    count = 0
    for index,value in enumerate(y_test):
        if value == y_pred[index]:
           count +=1
    acc = count/len(y_test)
    return acc
accuracy = accuracy_score(y_pred,y_valid)
print("M2 - ", accuracy * 100)

M2 -  51.0
